In [ ]:
#-*- coding: utf-8 -*-
# 라이브러리 import

try:
    from selenium import webdriver
except:
    !pip install selenium
    from selenium import webdriver

try:
    from webdriver_manager.chrome import ChromeDriverManager
except:
    !pip install webdriver_manager
    from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys

import datetime
from dateutil.relativedelta import relativedelta
try:
    from urllib import parse
except:
    !pip install urllib3
    from urllib import parse

import time

try:
    import csv
except:
    !pip install csv
    import csv

try:
    import getpass
except:
    !pip install getpass4
    import getpass

import re

In [ ]:
# 브라우저 옵션
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('start-maximized')
options.add_argument(
            "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
        )
options.add_argument("lang=ko_KR")

In [ ]:
# 비밀번호 마스킹 처리 함수
def printPass(password) :
    masking_password = ""
    for s in password :
        masking_password += "·"
    return masking_password

In [ ]:
# 로그인 처리 함수 loginType F = 페이스북 계정 로그인, I = 인스타 계정 로그인
def instaLogin(chrome, loginType="F", email = None, password = None) :
    chrome.get("https://www.instagram.com/accounts/login/")
    time.sleep(3)
    insta_formObj = chrome.find_element_by_css_selector("form[id=loginForm]")
    emailObj = None
    passObj = None
    loginBtnObj = None
    if loginType == "I":
        print("loginType = I")
        emailObj = insta_formObj.find_element_by_css_selector("input[name=username]")
        passObj = insta_formObj.find_element_by_css_selector("input[name=password]")
        loginBtnObj = insta_formObj.find_element_by_css_selector("button[type=submit]")
    else :
        print("loginType = F")
        loginMoveBtnObj = insta_formObj.find_element_by_css_selector("button.sqdOP.yWX7d.y3zKF[type=button]")
        loginMoveBtnObj.click()
        time.sleep(3)
        facebook_formObj = chrome.find_element_by_css_selector("form[id=login_form]")
        emailObj = facebook_formObj.find_element_by_css_selector("input[id=email]")
        passObj = facebook_formObj.find_element_by_css_selector("input[id=pass]")
        loginBtnObj = facebook_formObj.find_element_by_css_selector("button[id=loginbutton]")

    if email is None :
        email = input("이메일 또는 아이디 : ")
    else :
        print("이메일 또는 아이디 : " + email)

    if password is None :
        password = getpass.getpass("비밀번호 : ")
    else :
        print("비밀번호 : " + printPass(password))

    emailObj.clear()
    emailObj.send_keys(email)
    passObj.clear()
    passObj.send_keys(password)
    time.sleep(1)
    loginBtnObj.click()
    time.sleep(5)


In [ ]:
# 페이스북 로그인 처리 체크 후 재처리를 위한 함수
def loginFacebookCheck(chrome) :
    if chrome.current_url == "https://www.instagram.com/accounts/login/":
        insta_formObj = chrome.find_element_by_css_selector("form[id=loginForm]")
        loginMoveBtnObj = insta_formObj.find_element_by_css_selector("button.sqdOP.yWX7d.y3zKF[type=button]")
        loginMoveBtnObj.click()
        time.sleep(5)
        return True
    else :
        return False


In [ ]:
# 첫번째 사진 선택 함수
def selectFirst(chrome):
    try:
        first = chrome.find_element_by_css_selector("div._9AhH0")
        first.click()
        time.sleep(2)
        return True
    except Exception as e:
        print(e)
        return False


In [ ]:
# 다음 사진으로 이동 함수
def moveNext(chrome):
    try:
        right = chrome.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
        right.click()
        time.sleep(3)
        return True
    except Exception as e:
        print(e)
        return False


In [ ]:
# 내용 추출 함수
def getContent(chrome):
    # 본문 내용
    try:
        content_section= chrome.find_element_by_css_selector('article.M9sTE.L_LMM.JyscU.ePUX4')
        try:
            content = content_section.find_element_by_css_selector('div.C4VMK > span').text
        except Exception as e:
            print(e)
            content = ' '
        # 해시태그 
        tags = re.findall(r'#[^\s#,\\]+', content)  
        # 작성일자 
        date = content_section.find_element_by_css_selector('time._1o9PC.Nzb55').get_attribute('datetime')[:10]
        # 좋아요 
        try:
            like = content_section.find_element_by_css_selector('div.Nm9Fw > a > span').text   
        except Exception as e:
            print(e)
            like = 0
        # 위치
        try: 
            place = content_section.find_element_by_css_selector('div.M30cS').text
        except Exception as e:
            print(e)
            place = ''

        data = [content, date, like, place, tags]
        return data
    except Exception as e:
        print(e)
        return None

In [ ]:
# 크롤링 시작 함수
def startSearch(loginType = "F", email = None, password = None) :
    # 파일 인코딩 타입
    encode_type = "utf-8"
    chrome = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    try:
        instaLogin(chrome, loginType, email, password)
        time.sleep(2)
        
        # 겁색어 받기
        search_txt = input("search : ")
        search_encode = parse.quote(search_txt)

        #print(chrome.current_url)
        searchUrl = "https://www.instagram.com/explore/tags/" + search_encode + "/"
        
        chrome.get(searchUrl)
        time.sleep(3)
        
        # Facebook 로드인 연동이 안되는 경우가 있어 Facebook 재 연동처리
        if loginType == "F" :
            if loginFacebookCheck(chrome) :
                chrome.get(searchUrl)
                time.sleep(3)
        
        now_date = datetime.datetime.now()

        cvsFileName = "inta_search_" + search_txt + "_" + now_date.strftime("%Y-%m-%d") + ".csv"
        with open(cvsFileName, "w", newline="", encoding=encode_type) as f:
            colnames = ['content', 'date', 'like', 'place', 'tags']
            w = csv.writer(f)
            w.writerow(colnames)
        count = 0
        if selectFirst(chrome) :
            while True :
                count = count+1
                data = getContent(chrome)
                if data != None:
                    with open(cvsFileName, "a", newline="", encoding=encode_type) as f:
                        w = csv.writer(f)
                        print("[" + str(count) + "] : ", data)
                        w.writerow(data)
                if moveNext(chrome) :
                    time.sleep(2)
                else :
                    break;

    except Exception as e:
        print(e)
    finally:
        if chrome:
            chrome.close()

In [ ]:
startSearch()